# 데이터 품질 및 샘플 분포/편향 EDA
이 노트북에서는 데이터 구조, 샘플별 modality 매칭, class/카메라/배우/시나리오별 분포 및 편향을 분석합니다.

---
## 1. 필요 라이브러리 임포트

In [47]:
# 1. 필요 라이브러리 임포트
import os
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np
from pprint import pprint

## 2. 데이터 파일 경로 지정 및 폴더 구조 탐색
데이터 폴더 경로를 지정하고, 하위 폴더 및 파일 구조를 탐색하는 함수를 구현합니다.

In [48]:
# 2. 데이터 파일 경로 지정 및 폴더 구조 탐색
BASE_DIR = Path("../data/train/raw")
modalities = ["image", "video", "sensor"]
class_tokens = ["N", "SY"]

def list_leaf_dirs_with_files(path, exts=None):
    leaf_dirs = []
    if not path.exists():
        print(f"[경고] 경로 없음: {path.resolve()}")
        return leaf_dirs
    for root, dirs, files in os.walk(path):
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        files = [f for f in files if not f.startswith('.')]
        if exts is not None:
            files = [f for f in files if Path(f).suffix.lower() in exts]
        if files and not dirs:
            leaf_dirs.append((Path(root), files))
    return leaf_dirs

def detect_class_from_path(p: Path):
    label = None
    for part in p.parts:
        if part in class_tokens:
            label = part
    return label

exts_map = {
    "image": {".jpg", ".jpeg", ".png"},
    "video": {".mp4", ".avi", ".mov"},
    "sensor": {".csv"},
}

for m in modalities:
    base = BASE_DIR / m
    leaf_dirs = list_leaf_dirs_with_files(base, exts=exts_map.get(m))
    print(f"\n[{m}] leaf dirs count =", len(leaf_dirs))
    for leaf_dir, files in leaf_dirs[:3]:
        print(f"  샘플 폴더: {leaf_dir}, 파일 수: {len(files)}")


[image] leaf dirs count = 4128
  샘플 폴더: ../data/train/raw/image/N/N/00073_H_A_N_C8, 파일 수: 10
  샘플 폴더: ../data/train/raw/image/N/N/02604_H_A_N_C7, 파일 수: 10
  샘플 폴더: ../data/train/raw/image/N/N/02117_H_A_N_C3, 파일 수: 10

[video] leaf dirs count = 4128
  샘플 폴더: ../data/train/raw/video/N/N/00073_H_A_N_C8, 파일 수: 1
  샘플 폴더: ../data/train/raw/video/N/N/02604_H_A_N_C7, 파일 수: 1
  샘플 폴더: ../data/train/raw/video/N/N/02117_H_A_N_C3, 파일 수: 1

[sensor] leaf dirs count = 4128
  샘플 폴더: ../data/train/raw/sensor/N/N/00073_H_A_N_C8, 파일 수: 1
  샘플 폴더: ../data/train/raw/sensor/N/N/02604_H_A_N_C7, 파일 수: 1
  샘플 폴더: ../data/train/raw/sensor/N/N/02117_H_A_N_C3, 파일 수: 1

[sensor] leaf dirs count = 4128
  샘플 폴더: ../data/train/raw/sensor/N/N/00073_H_A_N_C8, 파일 수: 1
  샘플 폴더: ../data/train/raw/sensor/N/N/02604_H_A_N_C7, 파일 수: 1
  샘플 폴더: ../data/train/raw/sensor/N/N/02117_H_A_N_C3, 파일 수: 1


## 3. 샘플 센서 데이터프레임 로드 및 구조 확인
샘플 센서 데이터 파일을 pandas로 읽어와 데이터프레임의 head, shape, info를 출력합니다.

In [49]:
# 3. 샘플 센서 데이터프레임 로드 및 구조 확인
sample_sensor_path = '../data/train/raw/sensor/N/00002_H_A_N_C1/00002_H_A_N_C1.csv'
if os.path.exists(sample_sensor_path):
    df = pd.read_csv(sample_sensor_path)
    display(df.head())
    print('Shape:', df.shape)
    print(df.info())
else:
    print('샘플 센서 데이터 파일이 존재하지 않습니다.')

샘플 센서 데이터 파일이 존재하지 않습니다.


## 4. 컬럼별 데이터 타입 및 결측치 확인
데이터프레임의 각 컬럼별 데이터 타입과 결측치 개수를 확인합니다.

In [50]:
# 4. 컬럼별 데이터 타입 및 결측치 확인
if 'df' in locals():
    print(df.dtypes)
    print('결측치 개수:')
    print(df.isnull().sum())

## 5. 파일 매칭 및 샘플별 파일 집계
raw 데이터의 modality별(이미지, 비디오, 센서)로 샘플별 파일 존재 여부를 탐색하고, class별 샘플 수를 집계합니다.

In [51]:
# 5. 파일 매칭 및 샘플별 파일 집계
result = defaultdict(lambda: defaultdict(set))
for m in modalities:
    base = BASE_DIR / m
    leaf_dirs = list_leaf_dirs_with_files(base, exts=exts_map.get(m))
    for leaf_dir, files in leaf_dirs:
        cls = detect_class_from_path(leaf_dir)
        if cls is None:
            continue
        sample_id = leaf_dir.name
        result[cls][m].add(sample_id)

for cls in class_tokens:
    print(f"\n=== 클래스: {cls} ===")
    for m in modalities:
        print(f"{m}: 샘플 수 = {len(result[cls][m])}")


=== 클래스: N ===
image: 샘플 수 = 2272
video: 샘플 수 = 2272
sensor: 샘플 수 = 2272

=== 클래스: SY ===
image: 샘플 수 = 1856
video: 샘플 수 = 1856
sensor: 샘플 수 = 1856


In [52]:
for cls in class_tokens:
    print(f"\n=== 클래스: {cls} ===")
    for m in modalities:
        print(f"{m}: 샘플 수 = {len(result[cls][m])}")
    intersection = result[cls]["image"] & result[cls]["video"] & result[cls]["sensor"]
    print(f"모든 modality 존재 샘플 수 = {len(intersection)}")
    print(f"교집합 샘플 리스트: {sorted(list(intersection))[:10]}")


=== 클래스: N ===
image: 샘플 수 = 2272
video: 샘플 수 = 2272
sensor: 샘플 수 = 2272
모든 modality 존재 샘플 수 = 2272
교집합 샘플 리스트: ['00002_H_A_N_C1', '00002_H_A_N_C2', '00002_H_A_N_C3', '00002_H_A_N_C4', '00002_H_A_N_C5', '00002_H_A_N_C6', '00002_H_A_N_C7', '00002_H_A_N_C8', '00006_H_A_N_C1', '00006_H_A_N_C2']

=== 클래스: SY ===
image: 샘플 수 = 1856
video: 샘플 수 = 1856
sensor: 샘플 수 = 1856
모든 modality 존재 샘플 수 = 1856
교집합 샘플 리스트: ['00004_H_A_SY_C1', '00004_H_A_SY_C2', '00004_H_A_SY_C3', '00004_H_A_SY_C4', '00004_H_A_SY_C5', '00004_H_A_SY_C6', '00004_H_A_SY_C7', '00004_H_A_SY_C8', '00008_H_A_SY_C1', '00008_H_A_SY_C2']


## 6. 클린 샘플 리스트 생성 및 저장
정상적으로 모든 modality가 존재하는 샘플만을 모아 DataFrame으로 만들고, csv로 저장합니다.

In [53]:
# 6. 클린 샘플 리스트 생성 및 저장 (N/N, Y/SY 구조 반영)
clean_rows = []
for cls in class_tokens:
    # 모든 modality가 존재하는 샘플만 추출
    valid_samples = result[cls]["image"] & result[cls]["video"] & result[cls]["sensor"]
    for sample_id in valid_samples:
        # N/N 또는 Y/SY 구조로 class 경로 생성
        if cls == "N":
            class_path = "N/N"
        elif cls == "SY":
            class_path = "Y/SY"
        else:
            class_path = cls  # 혹시 다른 클래스가 있을 경우 그대로 사용
        clean_rows.append({
            "sample_id": sample_id,
            "class": class_path,
            "image_dir": f"../data/train/raw/image/{class_path}/{sample_id}",
            "video_dir": f"../data/train/raw/video/{class_path}/{sample_id}",
            "sensor_dir": f"../data/train/raw/sensor/{class_path}/{sample_id}",
        })
df_clean = pd.DataFrame(clean_rows)
df_clean.to_csv("train_clean.csv", index=False)
df_clean

,sample_id,class,image_dir,video_dir,sensor_dir
0,02181_H_A_N_C1,N/N,../data/train/raw/image/N/N/02181_H_A_N_C1,../data/train/raw/video/N/N/02181_H_A_N_C1,../data/train/raw/sensor/N/N/02181_H_A_N_C1
1,02117_H_A_N_C6,N/N,../data/train/raw/image/N/N/02117_H_A_N_C6,../data/train/raw/video/N/N/02117_H_A_N_C6,../data/train/raw/sensor/N/N/02117_H_A_N_C6
2,02497_H_A_N_C5,N/N,../data/train/raw/image/N/N/02497_H_A_N_C5,../data/train/raw/video/N/N/02497_H_A_N_C5,../data/train/raw/sensor/N/N/02497_H_A_N_C5
3,02604_H_A_N_C2,N/N,../data/train/raw/image/N/N/02604_H_A_N_C2,../data/train/raw/video/N/N/02604_H_A_N_C2,../data/train/raw/sensor/N/N/02604_H_A_N_C2
4,00322_H_A_N_C3,N/N,../data/train/raw/image/N/N/00322_H_A_N_C3,../data/train/raw/video/N/N/00322_H_A_N_C3,../data/train/raw/sensor/N/N/00322_H_A_N_C3
...,...,...,...,...,...
4123,00166_H_A_SY_C2,Y/SY,../data/train/raw/image/Y/SY/00166_H_A_SY_C2,../data/train/raw/video/Y/SY/00166_H_A_SY_C2,../data/train/raw/sensor/Y/SY/00166_H_A_SY_C2
4124,02226_H_A_SY_C6,Y/SY,../data/train/raw/image/Y/SY/02226_H_A_SY_C6,../data/train/raw/video/Y/SY/02226_H_A_SY_C6,../data/train/raw/sensor/Y/SY/02226_H_A_SY_C6
4125,02252_H_A_SY_C7,Y/SY,../data/train/raw/image/Y/SY/02252_H_A_SY_C7,../data/train/raw/video/Y/SY/02252_H_A_SY_C7,../data/train/raw/sensor/Y/SY/02252_H_A_SY_C7
4126,00579_H_D_SY_C7,Y/SY,../data/train/raw/image/Y/SY/00579_H_D_SY_C7,../data/train/raw/video/Y/SY/00579_H_D_SY_C7,../data/train/raw/sensor/Y/SY/00579_H_D_SY_C7


In [54]:
print(f"clean_rows 개수: {len(clean_rows)}")
df_clean = pd.DataFrame(clean_rows)
print(df_clean.head())

clean_rows 개수: 4128
        sample_id class                                   image_dir  \
0  02181_H_A_N_C1   N/N  ../data/train/raw/image/N/N/02181_H_A_N_C1   
1  02117_H_A_N_C6   N/N  ../data/train/raw/image/N/N/02117_H_A_N_C6   
2  02497_H_A_N_C5   N/N  ../data/train/raw/image/N/N/02497_H_A_N_C5   
3  02604_H_A_N_C2   N/N  ../data/train/raw/image/N/N/02604_H_A_N_C2   
4  00322_H_A_N_C3   N/N  ../data/train/raw/image/N/N/00322_H_A_N_C3   

                                    video_dir  \
0  ../data/train/raw/video/N/N/02181_H_A_N_C1   
1  ../data/train/raw/video/N/N/02117_H_A_N_C6   
2  ../data/train/raw/video/N/N/02497_H_A_N_C5   
3  ../data/train/raw/video/N/N/02604_H_A_N_C2   
4  ../data/train/raw/video/N/N/00322_H_A_N_C3   

                                    sensor_dir  
0  ../data/train/raw/sensor/N/N/02181_H_A_N_C1  
1  ../data/train/raw/sensor/N/N/02117_H_A_N_C6  
2  ../data/train/raw/sensor/N/N/02497_H_A_N_C5  
3  ../data/train/raw/sensor/N/N/02604_H_A_N_C2  
4  ../data/tr

## 7. 클래스 분포 및 비율 확인
클린 샘플 리스트에서 class별 샘플 수와 비율을 계산하여 출력합니다.

In [55]:
# 7. 클래스 분포 및 비율 확인
class_counts = df_clean['class'].value_counts()
class_ratio = df_clean['class'].value_counts(normalize=True)
print("=== Class Distribution ===")
print(class_counts)
print("\n=== Class Ratio ===")
print(class_ratio)

=== Class Distribution ===
class
N/N     2272
Y/SY    1856
Name: count, dtype: int64

=== Class Ratio ===
class
N/N     0.550388
Y/SY    0.449612
Name: proportion, dtype: float64


## 8. 클래스-카메라(Cx) 편향 분석
sample_id에서 카메라 번호를 추출하여 class별 카메라 분포와 비율을 집계합니다.

In [56]:
# 8. 클래스-카메라(Cx) 편향 분석
df_clean['camera'] = df_clean['sample_id'].str.extract(r'_C(\d+)$')
camera_class = df_clean.groupby(['class','camera']).size().unstack(fill_value=0)
camera_ratio = df_clean.groupby(['class','camera']).size().groupby(level=0).apply(lambda x: x / x.sum())
print("=== Class-Camera Count ===")
print(camera_class)
print("\n=== Class-Camera Ratio ===")
print(camera_ratio)

=== Class-Camera Count ===
camera    1    2    3    4    5    6    7    8
class                                         
N/N     284  284  284  284  284  284  284  284
Y/SY    232  232  232  232  232  232  232  232

=== Class-Camera Ratio ===
class  class  camera
N/N    N/N    1         0.125
              2         0.125
              3         0.125
              4         0.125
              5         0.125
              6         0.125
              7         0.125
              8         0.125
Y/SY   Y/SY   1         0.125
              2         0.125
              3         0.125
              4         0.125
              5         0.125
              6         0.125
              7         0.125
              8         0.125
dtype: float64


## 9. 클래스-배우(actor) 편향 분석
sample_id에서 배우 정보를 추출하여 class별 배우 분포와 비율을 집계합니다.

In [57]:
# 9. 클래스-배우(actor) 편향 분석
df_clean['actor'] = df_clean['sample_id'].str.extract(r'H_(A_[^_]+)_')
actor_class = df_clean.groupby(['class','actor']).size().unstack(fill_value=0)
actor_ratio = df_clean.groupby(['class','actor']).size().groupby(level=0).apply(lambda x: x / x.sum())
print(actor_class)
print(actor_ratio)

actor   A_N  A_SY
class            
N/N    1688     0
Y/SY      0  1288
class  class  actor
N/N    N/N    A_N      1.0
Y/SY   Y/SY   A_SY     1.0
dtype: float64


## 10. 클래스-시나리오(scene) 편향 분석
sample_id에서 시나리오 그룹을 추출하여 class별 시나리오 분포와 비율을 집계합니다.

In [58]:
# 10. 클래스-시나리오(scene) 편향 분석
df_clean['scene_group'] = df_clean['sample_id'].str.extract(r'^(\d{2})')
scene_class = df_clean.groupby(['class','scene_group']).size().unstack(fill_value=0)
scene_ratio = df_clean.groupby(['class','scene_group']).size().groupby(level=0).apply(lambda x: x / x.sum())
print(scene_class)
print(scene_ratio)

scene_group    00  01    02
class                      
N/N          1088  24  1160
Y/SY          848  24   984
class  class  scene_group
N/N    N/N    00             0.478873
              01             0.010563
              02             0.510563
Y/SY   Y/SY   00             0.456897
              01             0.012931
              02             0.530172
dtype: float64
